In [1]:
#Libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [2]:
import src.clean as src

In [3]:
#Some configuration for matplotlib and seaborn to look pretty
%matplotlib inline
%config Inlinebackend.figure_format= 'retina'
sns.set_context("poster")
sns.set(rc={"figure.figsize": (12.,6.)})
sns.set_style("whitegrid")

In [4]:
#Reading the clean Data Frame
df = pd.read_csv('Output/df_clean.csv')
df.drop(columns='Unnamed: 0', inplace=True)

In [5]:
df.sample()

,Country,Sex-ratio,Fertility,GDP per capita,Life expectancy,Meat consumption,Median age,Population growth,Suicide rate,Urbanization rate,Net Migration,Continent,HDI,Class
104,Kenya,1.0,3.5,4078.0,63.3,14.3,19.7,2.7,5.6,28.0,−0.2,Africa,0.59,In development


In [6]:
#Checking the amount of missing data over total. (77.9% complete)
print(f'The gathered data is {round((df.count().sum() - df.isna().sum().sum())*100/(df.shape[0]*df.shape[1]),1)}% complete')

The gathered data is 77.9% complete


In [7]:
# IMPORTANT: I have arbitrarily decided to drop countries which have less than 8 non-NaN values for a better analysis.
# This is due to missing over half of the given data.
df.dropna(thresh=8, inplace=True)
df.shape[0]

204

In [8]:
#We have dropped the small island Nations that didn't have an assigned Continent
df['Continent'].isna().sum()

0

In [9]:
#Checking the amount of missing data over total. We have improved 13pp (90.9% complete)
print(f'The gathered data is {round((df.count().sum() - df.isna().sum().sum())*100/(df.shape[0]*df.shape[1]),1)}% complete')

The gathered data is 90.9% complete
